In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [4]:
user = open("test.txt", "r").read()

In [5]:
file_loc = ["romance", "action", "comedy"]

In [11]:
doc = []
raw_data = ""
for i in file_loc:
    for j in os.listdir(i):
        temp = open(i+"/"+j, "r").read()
        doc.append([temp, i])
        raw_data += temp


In [12]:
doc

[['Nikola Tesla, (born July 9/10, 1856, Smiljan, Austrian Empire [now in Croatia]—died January 7, 1943, New York, New York, U.S.), Serbian American inventor and engineer who discovered and patented the rotating magnetic field, the basis of most alternating-current machinery. He also developed the three-phase system of electric power transmission. He immigrated to the United States in 1884 and sold the patent rights to his system of alternating-current dynamos, transformers, and motors to George Westinghouse. In 1891 he invented the Tesla coil, an induction coil widely used in radio technology.\nTesla was from a family of Serbian origin. His father was an Orthodox priest; his mother was unschooled but highly intelligent. As he matured, he displayed remarkable imagination and creativity as well as a poetic touch.\n\nTraining for an engineering career, he attended the Technical University at Graz, Austria, and the University of Prague. At Graz he first saw the Gramme dynamo, which operate

In [13]:
raw_data

'Nikola Tesla, (born July 9/10, 1856, Smiljan, Austrian Empire [now in Croatia]—died January 7, 1943, New York, New York, U.S.), Serbian American inventor and engineer who discovered and patented the rotating magnetic field, the basis of most alternating-current machinery. He also developed the three-phase system of electric power transmission. He immigrated to the United States in 1884 and sold the patent rights to his system of alternating-current dynamos, transformers, and motors to George Westinghouse. In 1891 he invented the Tesla coil, an induction coil widely used in radio technology.\nTesla was from a family of Serbian origin. His father was an Orthodox priest; his mother was unschooled but highly intelligent. As he matured, he displayed remarkable imagination and creativity as well as a poetic touch.\n\nTraining for an engineering career, he attended the Technical University at Graz, Austria, and the University of Prague. At Graz he first saw the Gramme dynamo, which operated 

In [14]:
vecotorizer = TfidfVectorizer()

In [17]:
vecotorizer.fit([raw_data])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [20]:
import numpy as np
data = np.array(doc)

In [23]:
import pandas as pd
data = pd.DataFrame({"feature": data[:,0], "label": data[:,1]})

In [24]:
data.head()

,feature,label
0,"Nikola Tesla, (born July 9/10, 1856, Smiljan, ...",romance
1,"Born in Germany 1879, Albert Einstein is one o...",action
2,Darwin’s theory of evolution declared that spe...,comedy


In [42]:
en = data["label"].unique()

In [47]:
record = {"romance":0, "action":1, "comedy":2}

In [48]:
# en = en.values
data["label"].replace(record.keys(), record.values(), inplace = True)

In [49]:
data.head()

,feature,label
0,"Nikola Tesla, (born July 9/10, 1856, Smiljan, ...",0
1,"Born in Germany 1879, Albert Einstein is one o...",1
2,Darwin’s theory of evolution declared that spe...,2


In [50]:
train = vecotorizer.transform(data["feature"]).toarray()

In [51]:
query = vecotorizer.transform([user]).toarray()

In [54]:
from sklearn.neighbors import NearestNeighbors
clf_euclidean = NearestNeighbors(n_neighbors=2)
clf_euclidean.fit(train, data["label"])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=1.0)

In [57]:
ep = clf_euclidean.kneighbors(query)

In [56]:
data["label"]

0    0
1    1
2    2
Name: label, dtype: int64

In [60]:
ep[1]

array([[0, 1]], dtype=int64)

In [62]:
from sklearn.neighbors import NearestNeighbors
clf_cosine = NearestNeighbors(n_neighbors=2,metric= "cosine")
clf_cosine.fit(train, data["label"])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=1.0)

In [63]:
ep2 = clf_cosine.kneighbors(query)

In [64]:
ep2

(array([[0.17140568, 0.28726812]]), array([[0, 1]], dtype=int64))

In [65]:
from sklearn.neighbors import NearestNeighbors
clf_jaccard = NearestNeighbors(n_neighbors=2, metric= "jaccard")
clf_jaccard.fit(train, data["label"])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='jaccard',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=1.0)

In [66]:
ep3 = clf_jaccard.kneighbors(query)

In [67]:
ep3

(array([[0.73295455, 0.82727273]]), array([[0, 1]], dtype=int64))

In [84]:
ans = [ep[1][0][0], ep2[1][0][0], ep3[1][0][0]]
sim = [ep[0][0][0], ep2[0][0][0], ep3[0][0][0]]

In [85]:
ans

[0, 0, 0]

In [86]:
sim

[0.5855009551992508, 0.17140568426961722, 0.7329545454545454]

In [89]:
def answer(ans):
    count_dic = ans[0]
    cnt = ans.count(ans[0])
    idx = 0
    for i in ans:
        if cnt<ans.count(i):
            count_dic = i
            cnt = ans.count(i)
    return count_dic


In [90]:
answer(ans)

0

In [93]:
rev_record ={i:j for j,i in record.items()}

In [94]:
rev_record

{0: 'romance', 1: 'action', 2: 'comedy'}